1 总结name_scope与variable_scope的作用以及异同点。

In [5]:
import tensorflow as tf

In [ ]:
name_scope可以为其作用域中的节点的name添加一个或多个前缀，并使用这些前缀作为划分内部与外部op范围的标记。

In [6]:

with tf.name_scope('scope_name') as name:
    print(name)
    b=tf.constant(1,name='const')
    print(b,name)

scope_name_1/
Tensor("scope_name_1/const:0", shape=(), dtype=int32) scope_name_1/


In [11]:
#在同一个name_scope的作用域中，填写name相同的Op，TensorFlow会自动加后缀
with tf.name_scope('scope_name') as name:
    a1 = tf.constant(1, name='const')
    print(b.name) 
    a2 = tf.constant(1, name='const')
    print(a2.name)  
#同名name_scope 自动加‘_1’

#我们可以使用多个name_scope，并且可以填写相同name的两个或多个name_scope，但TensorFlow会自动给name_scope的 name加上后缀：
with tf.name_scope('my_name') as name1:
      print(name1)  # >> my_name/
    
with tf.name_scope('my_name') as name2:
      print(name2)  #>> my_name_1/

scope_name_1/const:0
scope_name_4/const_1:0
my_name/
my_name_1/


In [19]:
#name_cope嵌套使用
#同级的name_scope
with tf.name_scope('name_t'):
    with tf.name_scope('name2') as name2:
        print(name2) 
#不同级的name_scope可以填入相同的name
with tf.name_scope('name_b') as name1:
    print(name1)  
    with tf.name_scope('name_b') as name2:
        print(name2)  

name_t/name2/
name_b/
name_b/name_b/


In [20]:
#从外部传入的Tensor，并不会在name_scope中加上前缀。
a = tf.constant(1, name='constt')
with tf.name_scope('my_name', values=[a]):
    print(a.name)  

constt:0


variable_scope
variable_scope主要用于管理变量作用域以及与变量相关的操作，同时variable_scope也可以像name_scope一样用来给不同操作区域划分范围

#使用详情
variable_scope(name_or_scope,  # 可以是name或者别的variable_scope
               default_name=None,
               values=None,
               initializer=None,  # 作用域中的变量默认初始化方法
               regularizer=None,  # 作用域中的变量默认正则化方法
               caching_device=None,  # 默认缓存变量的device
               partitioner=None,  # 用于应用在被优化之后的投影变量操作
               custom_getter=None,  # 默认的自定义的变量getter方法
               reuse=None,  # 变量重用状态
               dtype=None,  # 默认的创建变量的类型
               use_resource=None):  # 是否使用ResourceVariables代替默认的Variables

In [22]:
with tf.variable_scope('abc') as scope:
    a = tf.constant(1, name='const1')
    print(a.name) 

abc/const1:0


In [23]:
#创建同名的variable_scop
g = tf.Graph()
with g.as_default():
    with tf.variable_scope('abc') as scope:
        print(scope.name)  # >> abc
        n_scope = g.get_name_scope()
        print(n_scope)  # >> abc
        
    with tf.variable_scope('abc') as scope:
        print(scope.name)  # >> abc
        n_scope = g.get_name_scope()
        print(n_scope)  # >> abc_1

abc
abc
abc
abc_1


In [24]:
#variable_scope的最佳搭档是tf.get_variable()函数。
tf.get_variable('abc', dtype=tf.float32, shape=[])
#随机生成一个初始化的值，

<tf.Variable 'abc_1:0' shape=() dtype=float32_ref>

#tf.get_variable函数不能重名，否则会提示错误
a = tf.get_variable('abcd', shape=[1])
b = tf.get_variable('abcd', shape=[1])  
#ValueError: Variable abcd already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:


In [27]:
#tf.get_variable()的用法是随着reuse的状态而改变的
#reuse属性的值默认为none 这种情况创建的变量会加上前缀
#reuse属性值为true时候，利用name获取已经存在的变量

with tf.variable_scope('scope', reuse=None) as scope:
# 此时reuse=None，可以用来创建变量
    tf.get_variable('var', dtype=tf.float32, shape=[])
# 改修reuse=True
    scope.reuse_variables()
# 此时reuse=True，可以用来获得已有变量
    var = tf.get_variable('var')

In [28]:
#多级作用域
with tf.variable_scope('first_scope') as first_scope:
    print(first_scope.name)  # >> first_scope
    with tf.variable_scope('second_scope') as second_scope:
        print(second_scope.name)  # >> first_scope/second_scope
        print(tf.get_variable('var', shape=[1, 2]).name)  
        # >> first_scope/second_scope/var:0

first_scope
first_scope/second_scope
first_scope/second_scope/var:0


构建逻辑回归模型（只有模型部分，不包括训练部分），使用get_variable与variable_scope将变量的创建与使用分开

name_scope和variable_scope都可以为其作用域中的节点的name添加一个或多个前缀，并且可以再同级作用域和多级作用域中使用，
variable_scope有配合使用的get_variable，而name_scope没有，使用get_variable时，名字不能相同，否则系统会提示已经存在一个名字为XX的(名字相同)
variable_scope可以设置其作用域内的所有变量的一系列默认操作，比如初始化方法与正则化方法

构建逻辑回归模型（只有模型部分，不包括训练部分），使用get_variable与variable_scope将变量的创建与使用分开
构造预测函数

对于二分类问题，通常选择sigmoid函数作为基础逻辑函数，sigmoid函数的表达式为： 
sigmoid(z(i))=1/1+e−z(i)

对于单个输入 x(i)： 
z(i)=wTx(i)+b

可得到预测函数y^(i)为： 
y^(i)=a(i)=sigmoid(z(i))
对应的损失函数L(a(i),y(i))为： 
L(a(i),y(i))=−y(i)log(a(i))−(1−y(i))log(1−a(i))

然后通过对所有训练样例求和来计算代价函数： 
J=1/m∑i=1mL(a(i),y(i))

对于输入X，逻辑回归预测值
A=σ(wTX+b)=(a(1),a(2),...,a(m−1),a(m))
通过已知的训练数据与得到的预测值，可得到代价函数： 
J=−1/m∑i=1my(i)log(a(i))+(1−y(i))log(1−a(i))

反向传播 
dW=∂J/∂W=1/mX(A−Y)T
db=∂J∂b=1m∑i=1m(a(i)−y(i))
更新参数
W=W−α∗dW(10)
b=b−α∗db
决策域的选取
y^(i)=sigmoid(WTx(i)+b)


对应的决策函数为： 
y(i)p=1, if  y^(i)>0.5
y(i)p=0, if  y^(i)=<0.5
